<a href="https://colab.research.google.com/github/pranabIITM/CS6910/blob/main/PartA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D,Dense,Activation,MaxPooling2D,Flatten,BatchNormalization,Dropout,InputLayer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
def create_cnn_model(filters,fil_size,dense_neuron,batch_norm,dropout,padding,lr):
    keras.backend.clear_session()
    model = Sequential()
    model.add(InputLayer(input_shape=(227,227,3))) 

    for i in range(5):
        model.add(Conv2D(filters[i],fil_size[i],padding="same"))
        if batch_norm:
            model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        
    model.add(Flatten())
    model.add(Dense(dense_neuron))
    model.add(Dropout(dropout))
    model.add(Activation("relu"))

    model.add(Dense(10,activation='softmax'))
    adam = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

    return model

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE = 128

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.1,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


 
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DL_DATA_iNat/train',
    subset='training',
    target_size=(227,227),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle = True,
    seed = 45)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DL_DATA_iNat/train',
    subset = 'validation',
    target_size=(227,227),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle = True,
    seed = 45)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/DL_DATA_iNat/val',
    target_size=(227,227),
    batch_size=BATCH_SIZE,
    class_mode='categorical')          
    

Found 9018 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2009 images belonging to 10 classes.


In [31]:
fil=[32,64,64,128,128]
fils=[(7,7),(7,7),(7,7),(7,7),(7,7)]
d=0.15
batch_norm=True
epochs=5,
d_n=64
ler=0.001
trial = create_cnn_model(fil,fils,d_n,batch_norm,d,"same",ler)

In [ ]:
trial.fit(
          train_generator,
          steps_per_epoch = train_generator.samples // 256,
          validation_data = val_generator, 
          validation_steps = val_generator.samples // 256,
          epochs = 5
        )

In [ ]:
trial.evaluate(test_generator,batch_size = 32)

In [20]:
sweep_config = {
    'method': 'random', 
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': { 'values': [8]
        },
        'filters': {
            'values': [[16,32,64,128,256],
                       #[32,64,128,256,512],
                       #[512,256,128,64,32],
                       [128,64,32,16,8],
                       [64,64,64,64,64],
                       [32,32,32,32,32],
                       [48,48,48,48,48]
                      ]
        },
        'filter_size': {
            'values': [[(7,7),(7,7),(7,7),(7,7),(7,7)],
                       [(5,5),(5,5),(5,5),(5,5),(5,5)],
                       [(3,3),(3,3),(3,3),(3,3),(3,3)],
                       [(2,2),(2,2),(2,2),(2,2),(2,2)]
                      ]
        },
        'dense_neuron': {
            'values': [32,64,128,256]
        },
        'batch_norm': {
            'values': [True,False]
        },
        'dropout': {
            'values': [0.15,0.2]
        },
        'lr': {
            'values': [1e-4,1e-5]
        }
    }
}

In [9]:
!pip install wandb


     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 181 kB 44.8 MB/s 
     |████████████████████████████████| 144 kB 50.0 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=c5d7f4d3fe90f0a92856cb58f499011ca0960213bc2cad35d2e91fb491285d1d
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [11]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [22]:
sweep_id = wandb.sweep(sweep_config,project = 'cs6910-As-2')

Create sweep with ID: 3k0qgn8h
Sweep URL: https://wandb.ai/pranab045/cs6910-As-2/sweeps/3k0qgn8h


In [21]:
import wandb
from wandb.keras import WandbCallback

def train():
    config_defaults = {
        "filters": [32,64,64,128,128],
        "filter_size": [(7,7),(7,7),(7,7),(7,7),(7,7)],
        "dropout": 0.15,
        "batch_norm": True,
        "epochs": 5,
        "dense_neuron": 64,
        "lr": 0.001
    }
    wandb.init(config=config_defaults,name="CS6910-A2-P_A")
    config = wandb.config
    wandb.run.name = "flt_{}_fltsz_{}_do_{}_dsz_{}_lr_{}".format(config.filters,\
                                                      config.filter_size[0],\
                                                      config.dropout,\
                                                      config.dense_neuron,\
                                                      config.lr)
    wandb.run.save()

    model = create_cnn_model(config.filters,config.filter_size,config.dense_neuron,config.batch_norm,config.dropout,"same",config.lr)

    trained_arti = model.fit(
                              train_generator,
                              steps_per_epoch = train_generator.samples // BATCH_SIZE,
                              validation_data = val_generator, 
                              validation_steps = val_generator.samples // BATCH_SIZE,
                              epochs = config.epochs, 
                              callbacks=[WandbCallback(monitor='val_accuracy',mode='max')]
                            )
    model.evaluate(
                    test_generator,
                    batch_size = 32,
                    callbacks=[WandbCallback()]
                  )
    #model.save('/content/drive/MyDrive/DL_DATA_iNat/best_model.h5)
    wandb.finish()
    return model, trained_arti

In [23]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: q7a4mc2g with config:
wandb: 	batch_norm: False
wandb: 	dense_neuron: 32
wandb: 	dropout: 0.2
wandb: 	epochs: 8
wandb: 	filter_size: [[7, 7], [7, 7], [7, 7], [7, 7], [7, 7]]
wandb: 	filters: [48, 48, 48, 48, 48]
wandb: 	lr: 0.0001


Epoch 1/8
70/70 [==============================] - 2020s 29s/step - loss: 2.2829 - accuracy: 0.1324 - val_loss: 2.2528 - val_accuracy: 0.1808 - _timestamp: 1648477150.0000 - _runtime: 2027.0000
Epoch 2/8
70/70 [==============================] - 273s 4s/step - loss: 2.2245 - accuracy: 0.1810 - val_loss: 2.1851 - val_accuracy: 0.2087 - _timestamp: 1648477424.0000 - _runtime: 2301.0000
Epoch 3/8
70/70 [==============================] - 278s 4s/step - loss: 2.1805 - accuracy: 0.2088 - val_loss: 2.1745 - val_accuracy: 0.2031 - _timestamp: 1648477750.0000 - _runtime: 2627.0000
Epoch 4/8
70/70 [==============================] - 282s 4s/step - loss: 2.1645 - accuracy: 0.2202 - val_loss: 2.1466 - val_accuracy: 0.2154 - _timestamp: 1648478076.0000 - _runtime: 2953.0000
Epoch 5/8
70/70 [==============================] - 281s 4s/step - loss: 2.1444 - accuracy: 0.2279 - val_loss: 2.1353 - val_accuracy: 0.2321 - _timestamp: 1648478357.0000 - _runtime: 3234.0000
Epoch 6/8
70/70 [=====================

accuracy,▁▄▅▆▆▇██
epoch,▁▂▃▄▅▆▇█
loss,█▆▄▄▃▂▁▁
val_accuracy,▁▃▃▃▅▇█▇
val_loss,█▆▅▄▄▂▁▁
accuracy,0.25849
best_epoch,6
best_val_accuracy,0.28125
epoch,7
loss,2.08714
val_accuracy,0.27121


wandb: Agent Starting Run: 1to69osa with config:
wandb: 	batch_norm: True
wandb: 	dense_neuron: 256
wandb: 	dropout: 0.15
wandb: 	epochs: 8
wandb: 	filter_size: [[5, 5], [5, 5], [5, 5], [5, 5], [5, 5]]
wandb: 	filters: [128, 64, 32, 16, 8]
wandb: 	lr: 1e-05


Epoch 1/8



...

wandb: Agent Starting Run: tl52h5t7 with config:
wandb: 	batch_norm: True
wandb: 	dense_neuron: 32
wandb: 	dropout: 0.15
wandb: 	epochs: 8
wandb: 	filter_size: [[5, 5], [5, 5], [5, 5], [5, 5], [5, 5]]
wandb: 	filters: [128, 64, 32, 16, 8]
wandb: 	lr: 1e-05


Epoch 1/8



wandb: ERROR Run tl52h5t7 errored: ResourceExhaustedError()
wandb: Agent Starting Run: qyy0353p with config:
wandb: 	batch_norm: True
wandb: 	dense_neuron: 256
wandb: 	dropout: 0.15
wandb: 	epochs: 8
wandb: 	filter_size: [[2, 2], [2, 2], [2, 2], [2, 2], [2, 2]]
wandb: 	filters: [48, 48, 48, 48, 48]
wandb: 	lr: 0.0001


Epoch 1/8
70/70 [==============================] - 266s 4s/step - loss: 2.2948 - accuracy: 0.1877 - val_loss: 2.3471 - val_accuracy: 0.0938 - _timestamp: 1648482611.0000 - _runtime: 274.0000
Epoch 2/8
70/70 [==============================] - 259s 4s/step - loss: 2.0808 - accuracy: 0.2573 - val_loss: 2.4426 - val_accuracy: 0.1049 - _timestamp: 1648482928.0000 - _runtime: 591.0000
Epoch 3/8
 4/70 [>.............................] - ETA: 3:00 - loss: 2.0297 - accuracy: 0.2715

wandb: Ctrl + C detected. Stopping sweep.
